In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as skm
from sklearn.svm import SVC
import pickle

from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix
fhalf_scorer = make_scorer(fbeta_score, beta=0.5) 
from sklearn.metrics import precision_recall_curve
from scipy.signal import savgol_filter

# Model3 ('CG' bonds)

In [ ]:
data = np.concatenate((np.loadtxt('data4_r2_O21_c0d1_9x_700.csv',skiprows=1,delimiter=','),
                #    np.loadtxt('data4_r2_O21_c1_9x_900.csv',skiprows=1,delimiter=','),
                   np.loadtxt('data4_r2_O21_c0d1_9x_800.csv',skiprows=1,delimiter=','),

                   np.loadtxt('data4_r2_O21_c0d1_9x_750.csv',skiprows=1,delimiter=','),
                   np.loadtxt('data4_r2_O21_c0d1_9x_850.csv',skiprows=1,delimiter=','),
                #    np.loadtxt('data4_r2_O21_c0d1_9x_900.csv',skiprows=1,delimiter=','),

                   np.loadtxt('data4_r2_O21_c0d1_8x_800.csv',skiprows=1,delimiter=','),
                   np.loadtxt('data4_r2_O163_c0d1_9x_800.csv',skiprows=1,delimiter=','),
                   np.loadtxt('data4_r2_O341_c0d1_9x_800.csv',skiprows=1,delimiter=','),
                   np.loadtxt('data4_r2_O84_c0d1_9x_800.csv',skiprows=1,delimiter=','),
                   ))

In [ ]:
output = data[:,0] # data[:,1]==2
inputs = data[:,6:] # SOAP
scaler = StandardScaler()
scaler.fit(inputs)
X2 = scaler.transform(inputs)
y2 = output
Xtrain, Xtest, ytrain, ytest = train_test_split(X2,y2,test_size=0.2)

In [ ]:
# 5-fold cross validation 

gmean = []
fhalf = []
PR_AUC = []

for ic in np.logspace(-3,3,7):
    model0 = SVC(kernel='rbf',max_iter=1000000,C=ic) 
    # model0 = LogisticRegression(penalty='l2',max_iter=1000000,C=ic)
    fhalf_tmp = []
    PR_AUC_tmp = []
    gmean_tmp = []
    for icv in range(5):
        n_each = int(len(Xtrain)/5)
        idx_val = np.arange(len(Xtrain))[icv*n_each:(icv+1)*n_each]
        idx_train = np.setdiff1d(np.arange(len(Xtrain)),idx_val)
        model0.fit(Xtrain[idx_train],ytrain[idx_train])
        fhalf_tmp.append(fbeta_score(ytrain[idx_val],model0.predict(Xtrain[idx_val]),beta=0.5))
        precision, recall, thresholds = precision_recall_curve(ytrain[idx_val], model0.decision_function(Xtrain[idx_val]))
        PR_AUC_tmp = np.sum((precision[:-1]+precision[1:])/2*-np.diff(recall))
        tn, fp, fn, tp = confusion_matrix(ytrain[idx_val],model0.predict(Xtrain[idx_val])).ravel()
        Sensitivity = tp/(tp+fn)
        Specificity = tn/(fp+tn)
        gmean_tmp.append(np.sqrt(Sensitivity*Specificity))
    # cv_results = cross_validate(model0, Xtrain, ytrain, cv=5, scoring=fhalf_scorer)
    # accuracy.append(np.mean(cv_results['test_accuracy']))
    fhalf.append(np.mean(fhalf_tmp))
    PR_AUC.append(np.mean(PR_AUC_tmp))
    gmean.append(np.mean(gmean_tmp))

# accuracy = np.array(accuracy)
fhalf = np.array(fhalf)
PR_AUC = np.array(PR_AUC)
gmean = np.array(gmean)

print('CV best f0.5 = ', np.max(fhalf), 'and optimal C = ', np.logspace(-3,3,7)[np.argmax(fhalf)])
print('CV best AUC = ', np.max(PR_AUC), 'and optimal C = ', np.logspace(-3,3,7)[np.argmax(PR_AUC)])
print('CV gmean at best f0.5 = ', gmean[np.argmax(fhalf)], 'and optimal C = ', np.logspace(-3,3,7)[np.argmax(gmean)])

C_optimal = np.logspace(-3,3,7)[np.argmax(fhalf)]

In [ ]:
model = SVC(kernel='rbf',max_iter=1000000,C=C_optimal,probability=True) 
model.fit(X2,y2) # train the model using all data

# save the model 
with open('model3soap.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('scaler3soap.pkl','wb') as f:
    pickle.dump(scaler, f)